## Feature Engineering and CV based Winners' Solutions

continued from eda_4_29

New in this notebook

To-do: 
- number of active month
- More features related to history products

Features:
- eda_4_28
    - average of products for each (customer, product) pair
    - exponent weighted average of products each (customer, product) pair
    - time since presence of products, distance to the first 1
    - time to the last positive flank (01)
    - time to the last negative flank (10)
    - time to the last 1, to the nearest product purchase
    - time to the first 1, to the first product purchase
- before eda_4_25
    - customer info in the second month
    - products in the first month
    - combination of first and second month `ind_actividad_cliente`
    - combination of first and second month `tiprel_1mes`
    - combination of first month product by using binary number (`target_combine`)
    - encoding `target_combine` with 
        - mean number of new products
        - mean number of customers with new products
        - mean number of customers with each new products
    - Count patterns in the last `max_lag` months
    - Number of month to the last time the customer purchase each product
        - CV@2015-12-28: mlogloss=1.29349
        - Private score: 0.0302475, public score: 0.0299266
- eda_4_25
    - Use all available history data
        - E.g., for 2016-05-28 train data, use all previous months, for 2015-02-28, use 1 lag month. 
        - Need to create test set that use the same amount of previous months for each training data set. 
        - This is from [the second winner's solution](https://www.kaggle.com/c/santander-product-recommendation/discussion/26824), his bold part in paragraph 4.
    - Combine models trained on 2016-05-28 and 2015-06-28:
        - Private score: 0.0304583, public score: 0.0300839
        - This is to catch both seasonality and trend, presented in 2015-06-28 and 2016-05-28, respectively. 
        - This idea is mentioned by many winners, like [11-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823) and [14-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26808)

- eda_4_27
    - put 2015-06-28 and 2016-05-28 in the same data set, with the same lag=5
        - Private score:0.0303096, public score: 0.0299867
        - Different as [11-th winner's discussion](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823)
            > We tested this by adding 50% of May-16 data to our June model and sure enough, we went from 0.0301 to 0.0303. Then, we built separate models for Jun and May, but the ensemble didn’t work. We weren’t surprised because June data is better for seasonal products, and May data is better for trend products. And vice-versa, June data is bad for trend products and May data is bad for seasonal products. So, they sort of cancelled each other out.

        - But my score is always worse than theirs, maybe this is the reason why we have different observations

# Draft

In [1]:
from santander_helper import *

In [2]:
history = count_history('2016-02-28', 10)

In [4]:
month1 = '2016-02-28'
max_lag=10
    
month_new = month_list.index(month1)+1
month_end = month_list.index(month1)
month_start = month_end-max_lag+1

# Check if month_new is the last month
if month_new<len(month_list)-1:
    # Customers with new products in month_new
    customer_product_pair = pd.read_hdf('../input/customer_product_pair.hdf', 'customer_product_pair')
    ncodpers_list = customer_product_pair.loc[customer_product_pair.fecha_dato==month_list[month_new], 
        'ncodpers'].unique().tolist()

# Load data for all the lag related months
df = []
for m in range(month_start, month_end+1):
    df.append(pd.read_hdf('../input/data_month_{}.hdf'.format(month_list[m]), 'data_month'))

# concatenate data
df = pd.concat(df, ignore_index=True)

# Drop irrelevant features, which never occur in models
df = df.loc[:, ['fecha_dato']+cat_cols+target_cols]
# If month_new is not the last month, keep only customers with new products 
if month_new<len(month_list)-1:
    # select customers if this is not test set
    df = df.loc[df.ncodpers.isin(ncodpers_list), :]

# set ncodpers and fecha_dato as index
df.set_index(['ncodpers', 'fecha_dato'], inplace=True)

# unstack to make month as columns, now we have MultiIndex with months and 
# products in two levels
df = df.unstack(level=-1, fill_value=np.nan)

In [40]:
def find_first_valid_active_month(x, month1):
    '''Calculate the first valid month for each customer, x is the whole unstacked DataFrame'''
    # Valid: the records are not NAN
    # Active: buys any product
    
    # Use all the target features to determine if a customer is valid or active
    valid_active = x[target_cols].isnull().sum(axis=1, level=1)
    # First valid month: the first month that does not have any NAN, represented by index in month_list
    valid_active = pd.DataFrame(valid_active.apply(lambda u: month_list.index((u==0.0).idxmax()), axis=1))
    valid_active.columns = ['first_valid_month']
    valid_active['dist_first_valid_month'] = month_list.index(month1)-valid_active['first_valid_month']+1
    
    # First active month: the first month that buys products
    activity = x[target_cols].sum(axis=1, level=1)
    valid_active['active_percentage'] = (activity>=1.0).sum(axis=1)/activity.shape[1]
    
    return valid_active

In [41]:
a = find_first_valid_active_month(df, month1)

In [3]:
history.head()

,ind_cco_fin_ult1_dist_last_one,ind_cder_fin_ult1_dist_last_one,ind_cno_fin_ult1_dist_last_one,ind_ctju_fin_ult1_dist_last_one,ind_ctma_fin_ult1_dist_last_one,ind_ctop_fin_ult1_dist_last_one,ind_ctpp_fin_ult1_dist_last_one,ind_dela_fin_ult1_dist_last_one,ind_ecue_fin_ult1_dist_last_one,ind_fond_fin_ult1_dist_last_one,...,ind_nomina_ult1_dist_neg_flank_first,ind_plan_fin_ult1_dist_neg_flank_first,ind_pres_fin_ult1_dist_neg_flank_first,ind_reca_fin_ult1_dist_neg_flank_first,ind_recibo_ult1_dist_neg_flank_first,ind_tjcr_fin_ult1_dist_neg_flank_first,ind_valo_fin_ult1_dist_neg_flank_first,first_valid_month,dist_first_valid_month,active_percentage
ncodpers,,,,,,,,,,,,,,,,,,,,,
15889,0.0,10.0,10.0,10.0,10.0,10.0,0.0,10.0,10.0,10.0,...,10,10,10,10,10,2,10,4,10,1.0
15896,0.0,10.0,10.0,10.0,10.0,0.0,10.0,10.0,0.0,10.0,...,10,10,10,10,10,10,10,5,9,0.9
15900,10.0,10.0,1.0,10.0,10.0,0.0,10.0,10.0,10.0,10.0,...,10,10,10,10,10,10,10,4,10,1.0
15916,10.0,10.0,1.0,10.0,10.0,10.0,0.0,10.0,0.0,10.0,...,10,10,10,10,10,10,10,4,10,1.0
15925,0.0,10.0,10.0,10.0,10.0,0.0,10.0,10.0,10.0,10.0,...,10,10,10,10,1,2,10,4,10,1.0


In [39]:
df[target_cols].head().sum(axis=1, level=1)

fecha_dato,2015-05-28,2015-06-28,2015-07-28,2015-08-28,2015-09-28,2015-10-28,2015-11-28,2015-12-28,2016-01-28,2016-02-28
ncodpers,,,,,,,,,,
15889,4.0,4.0,4.0,3.0,3.0,3.0,3.0,4.0,4.0,3.0
15896,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0
15900,4.0,4.0,4.0,4.0,4.0,3.0,3.0,4.0,4.0,3.0
15916,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,7.0
15925,3.0,4.0,3.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0


In [30]:
a = df[target_cols].isnull().sum(axis=1, level=1).head()
a.apply(lambda x: month_list.index((x==0.0).idxmax()), axis=1)

ncodpers
15889    4
15896    5
15900    4
15916    4
15925    4
dtype: int64

In [44]:
def first_valid_month(x):
    '''Calculate the first valid month for each customer, x is the whole unstacked DataFrame'''
    # Use all the features to determine if 
    a = x[target_cols].count(axis=1, level=1).replace(0, np.nan)
    # Find the first month that is not NAN, the month is represented by index in month_list
    a = a.apply(lambda u: month_list.index(u.first_valid_index()), axis=1)
    return a

In [45]:
a = first_valid_month(df)

In [51]:
b = df.xs(axis=1, level=0, key='canal_entrada').copy()

In [55]:
a.name = 'first_valid_month'

In [56]:
b.join(a)

,2015-05-28,2015-06-28,2015-07-28,2015-08-28,2015-09-28,2015-10-28,2015-11-28,2015-12-28,2016-01-28,2016-02-28,first_valid_month
ncodpers,,,,,,,,,,,
15889,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4
15896,NaN,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5
15900,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4
15916,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4
15925,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4
15927,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4
15929,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4
15937,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4
15944,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4


In [41]:
a = df[target_cols].count(axis=1, level=1).replace(0, np.nan)

In [49]:
df.head()

canal_entrada                                              \
fecha_dato    2015-05-28 2015-06-28 2015-07-28 2015-08-28 2015-09-28   
ncodpers                                                               
15889                5.0        5.0        5.0        5.0        5.0   
15896                NaN        5.0        5.0        5.0        5.0   
15900                5.0        5.0        5.0        5.0        5.0   
15916                5.0        5.0        5.0        5.0        5.0   
15925                5.0        5.0        5.0        5.0        5.0   

                                                                     ...      \
fecha_dato 2015-10-28 2015-11-28 2015-12-28 2016-01-28 2016-02-28    ...       
ncodpers                                                             ...       
15889             5.0        5.0        5.0        5.0        5.0    ...       
15896             5.0        5.0        5.0        5.0        5.0    ...       
15900             5.0        5.0        5.0        5.0        5.0    ...       
15916             5.0        5.0        5.0        5.0        5.0    ...       
15925             5.0        5.0        5.0        5.0        5.0    ...       

           ind_valo_fin_ult1                                              \
fecha_dato        2015-05-28 2015-06-28 2015-07-28 2015-08-28 2015-09-28   
ncodpers                                                                   
15889                    1.0        1.0        1.0        1.0        1.0   
15896                    NaN        0.0        0.0        0.0        0.0   
15900                    0.0        0.0        0.0        0.0        0.0   
15916                    1.0        1.0        1.0        1.0        1.0   
15925                    0.0        0.0        0.0        0.0        0.0   

                                                                   
fecha_dato 2015-10-28 2015-11-28 2015-12-28 2016-01-28 2016-02-28  
ncodpers                                                           
15889             1.0        1.0        1.0        1.0        1.0  
15896             0.0        0.0        0.0        0.0        0.0  
15900             0.0        0.0        0.0        0.0        0.0  
15916             1.0        1.0        1.0        1.0        1.0  
15925             0.0        0.0        0.0        0.0        0.0  

[5 rows x 370 columns]

In [34]:
a.apply(lambda x: month_list.index(x.first_valid_index()), axis=1)

ncodpers
15889       4
15896       5
15900       4
15916       4
15925       4
15927       4
15929       4
15937       4
15944       4
15965       4
15975       4
15984       6
15998       4
16007       4
16152       4
16240       4
16268       4
16287       4
16357       4
16515       4
16532       4
16533       4
16617       4
16629       4
16641       4
16935       4
17065       4
17176       4
17211       4
17227       4
           ..
1535887    13
1535888    13
1535889    13
1535891    13
1535892    13
1535893    13
1535894    13
1535895    13
1535896    13
1535897    13
1535902    13
1535903    13
1535904    13
1535906    13
1535907    13
1535908    13
1535909    13
1535911    13
1535913    13
1535914    13
1535915    13
1535916    13
1535918    13
1535920    13
1535923    13
1535924    13
1535925    13
1535926    13
1535928    13
1535929    13
Length: 27770, dtype: int64

In [32]:
a.iloc[1, :].first_valid_index()

'2015-06-28'

In [23]:
a.groupby()

15889

In [9]:
df.ind_cco_fin_ult1.count(axis=1)

ncodpers
15889      10
15896       9
15900      10
15916      10
15925      10
15927      10
15929      10
15937      10
15944      10
15965      10
15975      10
15984       8
15998      10
16007      10
16152      10
16240      10
16268      10
16287      10
16357      10
16515      10
16532      10
16533      10
16617      10
16629      10
16641      10
16935      10
17065      10
17176      10
17211      10
17227      10
           ..
1535887     1
1535888     1
1535889     1
1535891     1
1535892     1
1535893     1
1535894     1
1535895     1
1535896     1
1535897     1
1535902     1
1535903     1
1535904     1
1535906     1
1535907     1
1535908     1
1535909     1
1535911     1
1535913     1
1535914     1
1535915     1
1535916     1
1535918     1
1535920     1
1535923     1
1535924     1
1535925     1
1535926     1
1535928     1
1535929     1
Length: 27770, dtype: int64